In [1]:
!pip install transformers datasets faiss-cpu sentence-transformers wikipedia-api streamlit

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=77f04cdd73dad3bd92d456530f1552681ca17a220a8f28c61f37a6dcbab8f918
 

In [2]:
from datasets import load_dataset

# Load a small dataset (Daily Dialog)
dataset = load_dataset("daily_dialog", split='train')

# Display the first sample
print(dataset[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

daily_dialog.py:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}


In [3]:
# Extract the dialogue text
dialogs = dataset['dialog']

# For simplicity, let's take only the first 5 dialogues
dialogs_sample = dialogs[:5]

# Split the dialogues into chunks (this will help in efficient retrieval)
chunks = []
for dialog in dialogs_sample:
    for turn in dialog:
        chunks.append(turn)

# Now we have a list of chunks
print(f"Total Chunks: {len(chunks)}")
print("First 3 Chunks:")
print(chunks[:3])


Total Chunks: 34
First 3 Chunks:
['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ']


In [4]:
import re

# Step 1: Lowercasing
chunks = [chunk.lower() for chunk in chunks]

# Step 2: Removing punctuation
chunks = [re.sub(r'[^\w\s]', '', chunk) for chunk in chunks]

# Step 3: Display the first 3 cleaned chunks to see the result
print(f"Total Chunks: {len(chunks)}")
print("First 3 Chunks after Preprocessing:")
print(chunks[:3])


Total Chunks: 34
First 3 Chunks after Preprocessing:
['say  jim  how about going for a few beers after dinner  ', ' you know that is tempting but is really not good for our fitness  ', ' what do you mean  it will help us to relax  ']


In [5]:
!pip install faiss-cpu
!pip install transformers


In [6]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Function to convert text to embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling
    return embeddings

# Get embeddings for all chunks
chunk_embeddings = get_embeddings(chunks)

# Display shape of embeddings to verify
print(f"Embeddings Shape: {chunk_embeddings.shape}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Embeddings Shape: torch.Size([34, 768])


In [7]:
import faiss
import numpy as np

# Convert embeddings to numpy array for FAISS
embeddings_np = chunk_embeddings.numpy()

# Build the FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])  # L2 distance for similarity search
index.add(embeddings_np)  # Add embeddings to index

print("FAISS index built and embeddings added.")


FAISS index built and embeddings added.


In [8]:
# Function to retrieve the top-k most relevant chunks for a query
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = get_embeddings([query]).numpy()  # Convert query to embedding
    distances, indices = index.search(query_embedding, top_k)  # Find nearest neighbors
    relevant_chunks = [chunks[idx] for idx in indices[0]]  # Get corresponding chunks
    return relevant_chunks

# Test the retrieval system with a query
query = "What do you think about fitness?"
relevant_chunks = retrieve_relevant_chunks(query, top_k=3)

# Display the top 3 relevant chunks
print("Top 3 Relevant Chunks:")
for i, chunk in enumerate(relevant_chunks):
    print(f"{i + 1}. {chunk}")


Top 3 Relevant Chunks:
1.  what do you mean  it will help us to relax  
2.  do you really think so  i dont  it will just make us fat and act silly  remember last time  
3.  you know that is tempting but is really not good for our fitness  


In [19]:
pip install transformers


In [20]:
# Concatenate the relevant chunks into one string
context = " ".join(relevant_chunks)

# Display the concatenated context
print("Context for Response Generation:")
print(context)


Context for Response Generation:
you know that is tempting but is really not good for our fitness what do you mean it will help us to relax


In [21]:
from transformers import pipeline

# Initialize the text generation pipeline with a pre-trained model
generator = pipeline('text-generation', model='distilgpt2')  # You can replace with 'EleutherAI/gpt-neo-1.3B' for a larger model

# Generate a response from the concatenated context
response = generator(context, max_length=150, num_return_sequences=1)

# Display the generated response
print("\nGenerated Response:")
print(response[0]['generated_text'])


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Response:
you know that is tempting but is really not good for our fitness what do you mean it will help us to relax and enjoy life at the beach for awhile, or for something with a few days at the beach for just a few days? And I hope it is not helpful for us to have sex or sleep but this is only for us with our partners.
